<a href="https://colab.research.google.com/github/BriacLEGUILLOU/Impact_score/blob/master/Impact_score_pr%C3%A9paration_des_donn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

# Load of bundle
chemin = '/content/gdrive/My Drive/Impact Score/Données partagées pour Impact Score/Autre données/'
bundle_raw = pd.read_csv(chemin + 'bundle.csv', sep=';', header=None,  index_col=False, names =['siren', 'year', 'accountability', 'bundle','amount'])
bundle_raw.head()

,siren,year,accountability,bundle,amount
0,5420120,2016,0,10,781843
1,5420120,2016,0,1,11836
2,5420120,2016,0,13,27644
3,5420120,2016,0,15,866430
4,5420120,2016,0,16,214907


# 1 Mise en forme des données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
pd.set_option('display.max_rows', 60)

In [ ]:
def obtention_log10(val):
  """ Permet de convertir un nombre de la façon suivante : 100000 devient 5 et -100 devient -2 """
  if val == np.nan: return np.nan
  elif (isinstance(val, float)) or (isinstance(val, int)):
    signe, nb = np.sign(val), np.absolute(val)
    if nb < 1: return 0
    else : return math.log10(nb) * signe
  else: return val

def filtre(df, col, filtre):
  """ Renvoie les entreprises contenant le filtre dans une colonne """
  return df[col].str.contains(filtre, case=False, na=False)

def division_colonnes(num, den):
  """ Cette fonction permet de diviser 2 colonnes """

  if (num == 0): # numérateur nul
    return 0
  elif den != 0:  # attention aux divisions par zéro
    return num / den
  else :
    return np.nan

def pos_neg(valeur):
  """ Renvoie la positif ou négatif pour la colonne sélectionnée """
  if (isinstance(valeur, float)) or (isinstance(valeur, int)):
    if valeur > 0:
      return '>0'
    elif valeur < 0:
      return '<0'
    else:
      return '=0'
  return ''

In [ ]:
# Identification des doublons
# bundle_raw[bundle_raw.duplicated(subset=['siren', 'year', 'accountability', 'bundle'], keep='first')]

In [ ]:
# On sélectionne uniquement les CR annuel complet, et on élimine certains doublons
bundle_raw = bundle_raw.loc[bundle_raw['accountability'] == 0,:].drop_duplicates(subset=['siren', 'year', 'accountability', 'bundle'], keep='first')
bundle_raw.head()

,siren,year,accountability,bundle,amount
0,5420120,2016,0,10,781843
1,5420120,2016,0,1,11836
2,5420120,2016,0,13,27644
3,5420120,2016,0,15,866430
4,5420120,2016,0,16,214907


In [ ]:
# On charge le nom des colonnes
ontology = pd.read_json('http://api.enthic.fr/company/ontology')

# Création d'un dataframe contenant les colonnes de chaque type de bilan
df_col = pd.DataFrame(columns=[ontology.loc[i,:].iloc[0].get("1") for i in range(4)]+
                      ['liasse '+ ontology.loc[i,:].iloc[0].get("1") for i in range(4)],
                               index=range(68))

# Lecture de tous les colonnes du compte de résultat complet
for type_bilan in range(4):
    dictionnaire = ontology.loc[type_bilan,:].iloc[0].get("code")
    nb_col = len(dictionnaire)
    
    for i in range(nb_col):
        df_col.iloc[i,type_bilan] = dictionnaire.get(str(i)).get("1")
        df_col.iloc[i,type_bilan+4] = dictionnaire.get(str(i)).get("0")
  
col_CR_complet = df_col.loc[:,['Compte annuel complet','liasse Compte annuel complet']]
col_CR_complet.to_csv(chemin + 'col_CR_complet.csv')
col_CR_complet

,Compte annuel complet,liasse Compte annuel complet
0,"Autres impôts, taxes et versements assimilés",9Z
1,Ventes de marchandises,FA
2,Production vendue biens,FD
3,Production vendue services,FG
4,Chiffres d’affaires nets,FJ
...,...,...
63,"Rétrocessions d’honoraires, commissions et cou...",YV
64,Taxe professionnelle,YW
65,Montant de la TVA. collectée,YY
66,Total TVA. déductible sur biens et services,YZ


In [ ]:
# On transforme la structure des données
bundle = pd.DataFrame(columns = ['year'] + list(range(0, 68)))
for year in sorted(bundle_raw['year'].unique()):
    bundle_year = bundle_raw.loc[bundle_raw['year']==year,:].pivot(index='siren', columns='bundle', values='amount')
    bundle_year['year'] = year
    bundle = pd.concat([bundle,bundle_year])


bundle = bundle.astype(float)

bundle.columns = ['year'] + list(col_CR_complet['Compte annuel complet'])
bundle['siren'] = bundle.index
print(bundle)
# La nouvelle forme des données est [111013 rows x 68 columns] = 7M de celulles
# L'ancienne forme des données était [3000000 rows x 5 columns] = 15M de cellules

             year  ...      siren
414381475  1919.0  ...  414381475
420210577  2007.0  ...  420210577
527710172  2008.0  ...  527710172
420210577  2009.0  ...  420210577
492952080  2009.0  ...  492952080
...           ...  ...        ...
998308415  2019.0  ...  998308415
998358303  2019.0  ...  998358303
998541502  2019.0  ...  998541502
998893408  2019.0  ...  998893408
998912703  2019.0  ...  998912703

[1294936 rows x 70 columns]


# 2. Chargement et préparation de ape, identity, ess

In [ ]:
# Chargement et préparation de APE
ape = pd.read_json('http://api.enthic.fr/company/ape').transpose()
ape.columns = ['ape', 'ape_name']
ape['ape_len'] = ape['ape'].apply(lambda x: len(x))

ape['ape_len'].value_counts()

5    732
4    562
3    290
2     89
Name: ape_len, dtype: int64

In [ ]:
# Définition des catégories des ape
def get_cat(string, num):
  """ permet d'obtenir la catégorie de l'APE"""
  string = str(string)
  assert isinstance(string, str)
  if num == 0:
    return string[0:2]
  elif len(string) > num+1:
    return string[num+1]
  else :
    return ''

for i in range(5):
  ape['ape_cat' + str(i)] = ape['ape'].apply(lambda x: get_cat(x, i))

ape['enthic'] = ape.index
ape

,ape,ape_name,ape_len,ape_cat0,ape_cat1,ape_cat2,ape_cat3,ape_cat4,enthic
0,1020Z,"Transform. & conserv. poisson, crustacés & mol...",5,10,2,0,Z,,0
1,812,Activités de nettoyage,3,81,2,,,,1
2,1610B,Imprégnation du bois,5,16,1,0,B,,2
3,0910Z,Activités de soutien à l'extraction d'hydrocar...,5,09,1,0,Z,,3
4,7420Z,Activités photographiques,5,74,2,0,Z,,4
...,...,...,...,...,...,...,...,...,...
1668,2051,Fabrication de produits explosifs,4,20,5,1,,,1668
1669,1431,Fabrication d'articles chaussants à mailles,4,14,3,1,,,1669
1670,9525,Réparation d'articles d'horlogerie et de bijou...,4,95,2,5,,,1670
1671,3211,Frappe de monnaie,4,32,1,1,,,1671


In [ ]:
# Chargement de identity
identity = pd.read_csv(chemin + 'identity.csv', sep=';', header=None,  index_col=False, names =['siren', 'denomination', 'enthic', 'postal_code','town'])

def numeriser(nombre):
  """ permet de numériser un nombre, si possible """
  try:
    num = int(nombre)
    return num
  except ValueError:
    return ''


identity['enthic'] = identity['enthic'].apply(numeriser)
identity

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,siren,denomination,enthic,postal_code,town
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS
...,...,...,...,...,...
1598335,999990302,AMUNDI OBLIG INTERNATIONALES,329,75015,PARIS
1598336,999990369,ECOFI INVESTISSEMENTS,1195,75009,PARIS 9
1598337,999990542,BARCLAYS DIVERSIFICATION,545,75012,PARIS
1598338,999990666,ASSU-VIE SOCIETE FRANCAISE D'ASSURANCES SUR LA...,909,75008,PARIS


In [ ]:
# Chargement de la liste des entreprises de l'ESS
df_ess = pd.read_csv(chemin + 'listeess_ldds_vjuin2020_ok_2.csv', sep=';', encoding='iso8859_16')
df_ess.columns = ['ess_' + x for x in df_ess.columns]
df_ess['ess'] = True
df_ess

,ess_N° IDENTITE ENTREPRISE (SIREN),ess_RAISON SOCIALE,ess_ENSEIGNE,ess_SIGLE,ess_CP VILLE,ess_FORME JURIDIQUE,ess
0,5650148,SCIC VALDURANCE HABITAT,NaN,NaN,04000 DIGNE LES BAINS,Autres coopératives,True
1,5720610,PICARDIE MARITIME HABITAT FONDATION PAUL DUCLERCQ,NaN,NaN,80100 ABBEVILLE,Autres coopératives,True
2,5781133,IDEA GROUPE,NaN,MTTM,44550 MONTOIR DE BRETAGNE,Sociétés coopératives et participatives (SCOP),True
3,5880679,SA COOPERATIVE D'INTERET COLLECTIF POURL'ACCES...,NaN,NaN,44570 TRIGNAC,Autres coopératives,True
4,6080170,COOP BREIZH,NaN,NaN,29540 SPEZET,Autres coopératives,True
...,...,...,...,...,...,...,...
265408,821664026,TOUT PRES DE CHEZ MOI,NaN,NaN,42110 Chambeon,Coopératives multisociétariales (SCIC),True
265409,832995799,UNIVERSAL AMBULANCES,NaN,NaN,84000 Avignon,Coopératives multisociétariales (SCIC),True
265410,843221326,UTOPIA SANTA MAGDALENA,NaN,NaN,31170 Tournefeuille,Sociétés coopératives et participatives (SCOP),True
265411,842368763,VITAE FORMATION,NaN,NaN,49120 Chemille,Coopératives multisociétariales (SCIC),True


for encoding in list(df_encoding.loc[19:,'Codec']):
  try:
    df_ess = pd.read_csv(chemin + 'listeess_ldds_vjuin2020_ok_2.csv', nrows=100, sep=';', encoding=encoding, error_bad_lines=False)
    print('réussi :', encoding)
    pass
  except UnicodeDecodeError:
    pass
  except UnicodeDecodeError:
    pass
  else:
    pass
  finally:
    pass

# 3. Jointure de bundle, ape et identity

In [ ]:
# Jointure de identity et de ape
identity_ape = identity.merge(ape, left_on='enthic', right_on='enthic', how='left')
identity_ape.head()

,siren,denomination,enthic,postal_code,town,ape,ape_name,ape_len,ape_cat0,ape_cat1,ape_cat2,ape_cat3,ape_cat4
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE,7010Z,Activités des sièges sociaux,5.0,70,1,0,Z,
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS,4778C,Autres commerces de détail spécialisés divers,5.0,47,7,8,C,
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS,5630Z,Débits de boissons,5.0,56,3,0,Z,
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU,1721A,Fabrication de carton ondulé,5.0,17,2,1,A,
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS,2030Z,"Fabrication de peintures, vernis, encres et ma...",5.0,20,3,0,Z,


In [ ]:
# Jointure de identity_ape et df_ess
identity_ape_ess = identity_ape.merge(df_ess, left_on='siren', right_on='ess_N° IDENTITE ENTREPRISE (SIREN)', how='left')
identity_ape_ess.head(5)

,siren,denomination,enthic,postal_code,town,ape,ape_name,ape_len,ape_cat0,ape_cat1,ape_cat2,ape_cat3,ape_cat4,ess_N° IDENTITE ENTREPRISE (SIREN),ess_RAISON SOCIALE,ess_ENSEIGNE,ess_SIGLE,ess_CP VILLE,ess_FORME JURIDIQUE,ess
0,5420120,STE DES SUCRERIES DU MARQUENTERRE,362,62140,MARCONNELLE,7010Z,Activités des sièges sociaux,5.0,70,1,0,Z,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5450093,SARL BAS ALPINE D AUTOCARS,273,04800,GREOUX LES BAINS,4778C,Autres commerces de détail spécialisés divers,5.0,47,7,8,C,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5450119,LE GRAND CAFE ET GASSENDI,1189,04000,DIGNE-LES-BAINS,5630Z,Débits de boissons,5.0,56,3,0,Z,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5520176,HERNAS CARTONNAGES,1300,80210,FEUQUIERES-EN-VIMEU,1721A,Fabrication de carton ondulé,5.0,17,2,1,A,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5520242,PROSPA,1566,80510,LONGPRE-LES-CORPS-SAINTS,2030Z,"Fabrication de peintures, vernis, encres et ma...",5.0,20,3,0,Z,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
identity_ape_ess['ess'].value_counts()

True    4836
Name: ess, dtype: int64

In [ ]:
# Jointure de identity_ape et bundle
data = bundle.merge(identity_ape_ess, left_on='siren', right_on='siren', how='left')
data['ess'] = data['ess'].fillna(False)
data.shape

(1294936, 89)

In [ ]:
data['ess'].value_counts()

False    1286674
True        8262
Name: ess, dtype: int64

# 4. Sauvegarde des données par catégorie

In [ ]:
#with open('/content/gdrive/My Drive/Impact Score/Export données par catégorie/data.xlsx', 'w') as f:
data.to_csv(chemin + 'all_data.csv')

In [ ]:
"""for j in ['0' + str(i) for i in range(0,10)]+ [str(i) for i in list(range(10,data['ape_cat0'].dropna().astype(int).max() +1))]:
  data[data['ape_cat0']==str(j)].to_excel('/content/gdrive/My Drive/Impact Score/Export données par catégorie/data{}.xlsx'.format(j))
  print(j)
  """